In [1]:
# Customer Purchase Behavior & Revenue Intelligence System  
## Notebook 01: Data Cleaning & Preprocessing  

### Objective  
#The objective of this notebook is to clean, preprocess, and prepare raw e-commerce transaction data for downstream analysis, including customer segmentation, revenue analysis, and dashboard development.

### Dataset Description  
#The dataset contains e-commerce transaction records including customer purchases, product details, order dates, quantities, and pricing information.


In [2]:
# Import Libraries

import pandas as pd 
import numpy as np 


In [3]:
# Load Dataset

file_path = "ecommerce_transactions_dataset.csv"
df = pd.read_csv(file_path)

# Preview data
df.head()

,OrderID,CustomerName,CustomerEmail,City,OrderDate,Product,Quantity,Discount(%),TotalPrice($),PaymentMethod,Shipped
0,bdd640fb-0667-4ad1-9c80-317fa3b1799d,Daniel Doyle,garzaanthony@example.org,East William,2024-04-29,Smartphone,3,5,2280.0,Credit Card,Yes
1,b38a088c-a65e-4389-b74d-0fb132e70629,Connie Lawrence,blakeerik@example.com,Joshuaborough,2025-04-05,Keyboard,4,0,400.0,Debit Card,Yes
2,6142ea7d-17be-4111-9a2a-73ed562b0f79,Ryan Munoz,blairamanda@example.com,New Kellystad,2024-11-17,Monitor,3,0,900.0,Debit Card,Yes
3,a9488d99-0bbb-4599-91ce-5dd2b45ed1f0,Dylan Miller,michellejames@example.com,Franciscostad,2024-07-05,Smartphone,1,5,760.0,Google Pay,Yes
4,aefcfad8-efc8-4849-b3aa-7efe4458a885,Lisa Jackson,frankgray@example.net,Lake Mark,2025-01-26,Smartphone,2,0,1600.0,Google Pay,Yes


In [4]:
# Initial Data Inspection

df.shape

(10000, 11)

In [5]:
df.columns

Index(['OrderID', 'CustomerName', 'CustomerEmail', 'City', 'OrderDate',
       'Product', 'Quantity', 'Discount(%)', 'TotalPrice($)', 'PaymentMethod',
       'Shipped'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   OrderID        10000 non-null  object 
 1   CustomerName   10000 non-null  object 
 2   CustomerEmail  10000 non-null  object 
 3   City           10000 non-null  object 
 4   OrderDate      10000 non-null  object 
 5   Product        10000 non-null  object 
 6   Quantity       10000 non-null  int64  
 7   Discount(%)    10000 non-null  int64  
 8   TotalPrice($)  10000 non-null  float64
 9   PaymentMethod  10000 non-null  object 
 10  Shipped        10000 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 859.5+ KB


In [7]:
df.isnull().sum()

OrderID          0
CustomerName     0
CustomerEmail    0
City             0
OrderDate        0
Product          0
Quantity         0
Discount(%)      0
TotalPrice($)    0
PaymentMethod    0
Shipped          0
dtype: int64

In [8]:
df.duplicated().sum()

np.int64(0)

In [9]:
## Data Cleaning

# Remove Duplicate Rows
df = df.drop_duplicates()

In [10]:
# Handle Missing Values

# Fill numerical missing values with median
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].median())

# Fill categorical missing values with mode
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])



In [11]:
## Convert order_date to datetime

df['OrderDate'] = pd.to_datetime(df['OrderDate'], errors='coerce')

# Ensure numeric columns are correct
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['TotalPrice($)'] = pd.to_numeric(df['TotalPrice($)'], errors='coerce')


In [12]:
## Feature Engineering

# Order Value (Already present but renamed cleanly)
df['order_value'] = df['TotalPrice($)']


In [13]:
# Data Features

df['order_year'] = df['OrderDate'].dt.year
df['order_month'] = df['OrderDate'].dt.month
df['order_month_name'] = df['OrderDate'].dt.month_name()


In [14]:
customer_agg = df.groupby('CustomerEmail').agg(
    customer_name=('CustomerName', 'first'),
    city=('City', 'first'),
    total_orders=('OrderID', 'nunique'),
    total_quantity=('Quantity', 'sum'),
    total_revenue=('order_value', 'sum'),
    avg_order_value=('order_value', 'mean'),
    first_purchase=('OrderDate', 'min'),
    last_purchase=('OrderDate', 'max')
).reset_index()

customer_agg.head()


,CustomerEmail,customer_name,city,total_orders,total_quantity,total_revenue,avg_order_value,first_purchase,last_purchase
0,aadams@example.net,Joseph Davidson,North Micheleshire,1,4,4800.0,4800.0,2024-05-06,2024-05-06
1,aaron25@example.org,Jay Barnes,Loritown,1,4,600.0,600.0,2024-12-05,2024-12-05
2,aaron31@example.com,Jennifer Bond,Joanneberg,1,2,228.0,228.0,2024-04-29,2024-04-29
3,aaron48@example.com,Alyssa Morris,New Kelly,1,2,380.0,380.0,2025-03-26,2025-03-26
4,aaron56@example.org,Susan Baker,Stoneview,1,3,285.0,285.0,2025-01-28,2025-01-28


In [15]:
df.to_csv("../data/processed/clean_transactions.csv", index=False)
customer_agg.to_csv("../data/processed/customer_summary.csv", index=False)


OSError: Cannot save file into a non-existent directory: '../data/processed'